## Семинар 1 Индекс

## Intro

In [71]:
import os
import pandas as pd
import numpy as np
import re

### работа с файлами и папками

In [32]:
curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [2]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному ти до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

C:\My\studies\HSE\prog\infosearch\1 Index\test.txt
test.txt
True


### os.listdir  
возвращает список файлов в данной директории

In [3]:
os.listdir(curr_dir)

['.ipynb_checkpoints', 'lec1_Index.pdf', 'sem1_Index.ipynb', 'test.txt']

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [4]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

C:\My\studies\HSE\prog\infosearch\1 Index\lec1_Index.pdf
C:\My\studies\HSE\prog\infosearch\1 Index\sem1_Index.ipynb
C:\My\studies\HSE\prog\infosearch\1 Index\test.txt


> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [5]:
list(os.walk(curr_dir))

[('C:\\My\\studies\\HSE\\prog\\infosearch\\1 Index',
  ['.ipynb_checkpoints'],
  ['lec1_Index.pdf', 'sem1_Index.ipynb', 'test.txt']),
 ('C:\\My\\studies\\HSE\\prog\\infosearch\\1 Index\\.ipynb_checkpoints',
  [],
  [])]

### чтение файла 

In [7]:
fpath = 'test.txt'


# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [1]:
### _check : в коллекции должно быть около 165 файлов

С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


## вспомним pandas и сделаем сырую базу текстов и мета

In [55]:
raw_data = pd.DataFrame()
raw_data = pd.DataFrame(columns=['name', 'season', 'episode', 'text'])
raw_data = raw_data.append({'name': '1', 'season': 2 , 'episode': 2, 'text': '2'}, ignore_index=True)
raw_data

,name,season,episode,text
0,1,2,2,2


In [56]:
raw_data = raw_data.drop(0)
raw_data

,name,season,episode,text


In [57]:
for root, dirs, files in os.walk(os.path.join(curr_dir, 'friends')):
    for name in files:
        ep_num, ep_name = name.split(' - ')[1:]
        ep_name = ep_name.strip('ru.txt')
        season, episode = ep_num.split('x')
        with open(os.path.join(root, name), 'r', encoding='utf-8') as f:
            text = f.read()
        raw_data = raw_data.append({'name': ep_name, 'season': season, 'episode': episode, 'text': text}, ignore_index=True)
raw_data.head()

,name,season,episode,text
0,The One Where Monica Gets A Roommate,1,01,﻿Друзья. Как все началось\n\nДа нечего рассказ...
1,The One With The Sonogram At The End,1,02,"﻿Вы, мужики, не понимаете\n\nДля нас поцелуй т..."
2,The One With The Thumb,1,03,"﻿- Привет, ребята.\n- Привет, Фиби..\n\n- Как ..."
3,The One With George Stephanopoulos,1,04,"﻿Так, теперь Фиби?\n\nЕсли бы я была всемогуще..."
4,The One With The East German Laundry Detergen,1,05,"﻿Может, хватит? Здесь нет ничего особенного\n\..."


## очистим тексты и сохраним результат

In [100]:
def split_strip(text):
    low = text.lower()
    stripped = re.sub('\.|,|#|$|%|\\|\'|\(|\)|-|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~', '', low)
    words = stripped.split()
    return words

In [101]:
split_strip('фирр Рт ыФРЫ - !!№ лфрсыор')

['фирр', 'рт', 'ыфры', '!!№', 'лфрсыор']